In [1]:
import pandas as pd
import os
import shutil
from pathlib import Path
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('jd.csv')
df.head(5)

,Unnamed: 0,job_title,work_method,contractor_tier,durations,earned_amount,apply_connect,list_tags,job_description,payment_type,hourly_rate,hourly_rate_min,hourly_rate_max,duration_time,hours_per_week
0,0,Tiktok followers and video likes,Hourly,Intermediate,"Less than 1 month, Less than 30 hrs/week",200,4.0,"Advertising, Graphic Design, YouTube Developme...",I opened an account and I’m trying to get it v...,Hourly,NaN,17.357546,40.797320,Less than 1 month,Less than 30 hrs/week
1,1,Mail Chimp Specialist to build a landing page ...,Fixed-price,Intermediate,"Less than 1 month, Less than 30 hrs/week",4000,4.0,"Social Media Lead Generation, Travel & Hospita...",Very straight forward job. Needing a landing p...,Fixed-price,NaN,29.077433,29.077433,Less than 1 month,Less than 30 hrs/week
2,2,Doodly expert needed,Hourly: $8-$15,Intermediate,"More than 6 months, 30+ hrs/week",100000,4.0,Advertisement,We need a doodle video created I can explain m...,Hourly,$8-$15,8.000000,15.000000,More than 6 months,30+ hrs/week
3,3,Media Buyer wanted for KETO Products,Hourly: $5-$8,Intermediate,"More than 6 months, Less than 30 hrs/week",300000,8.0,"Paid Media, Video Editing, Social Media Market...",Job Description: We are seeking an experienced...,Hourly,$5-$8,5.000000,8.000000,More than 6 months,Less than 30 hrs/week
4,4,Creative director for Facebook ads,Hourly,Intermediate,"Less than 1 month, Less than 30 hrs/week",30000,8.0,"Advertising, Advertising Strategy, Adobe Photo...",We are looking to hire a full time Creative Di...,Hourly,NaN,17.357546,40.797320,Less than 1 month,Less than 30 hrs/week


In [3]:
# drop all columns except title and description
df = df[['Unnamed: 0', 'job_title', 'job_description', 'list_tags', 'contractor_tier']]
df.head(3)

,Unnamed: 0,job_title,job_description,list_tags,contractor_tier
0,0,Tiktok followers and video likes,I opened an account and I’m trying to get it v...,"Advertising, Graphic Design, YouTube Developme...",Intermediate
1,1,Mail Chimp Specialist to build a landing page ...,Very straight forward job. Needing a landing p...,"Social Media Lead Generation, Travel & Hospita...",Intermediate
2,2,Doodly expert needed,We need a doodle video created I can explain m...,Advertisement,Intermediate


In [4]:
df['job_ids'] = df['Unnamed: 0']
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head()

,job_title,job_description,list_tags,contractor_tier,job_ids
0,Tiktok followers and video likes,I opened an account and I’m trying to get it v...,"Advertising, Graphic Design, YouTube Developme...",Intermediate,0
1,Mail Chimp Specialist to build a landing page ...,Very straight forward job. Needing a landing p...,"Social Media Lead Generation, Travel & Hospita...",Intermediate,1
2,Doodly expert needed,We need a doodle video created I can explain m...,Advertisement,Intermediate,2
3,Media Buyer wanted for KETO Products,Job Description: We are seeking an experienced...,"Paid Media, Video Editing, Social Media Market...",Intermediate,3
4,Creative director for Facebook ads,We are looking to hire a full time Creative Di...,"Advertising, Advertising Strategy, Adobe Photo...",Intermediate,4


In [5]:
with open('all_job_to_extract.txt') as f:
    lines = f.readlines()
job_ids = [int(line.strip()) for line in lines]
job_ids = list(set(job_ids))
print(job_ids[:10])
print(len(job_ids))

[0, 40961, 4098, 14339, 45060, 2053, 51209, 34826, 4107, 28681]
662


In [6]:
df = df[df['job_ids'].isin(job_ids)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 660 entries, 0 to 64201
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job_title        660 non-null    object
 1   job_description  660 non-null    object
 2   list_tags        648 non-null    object
 3   contractor_tier  660 non-null    object
 4   job_ids          660 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 30.9+ KB


# EDA

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import nltk

from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
import re
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from sklearn.metrics import classification_report, confusion_matrix

# nltk.download('punkt')
# nltk.download('stopwords')


In [8]:
# only take 2000 jd
# df = df.sample(2000)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 660 entries, 0 to 64201
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job_title        660 non-null    object
 1   job_description  660 non-null    object
 2   list_tags        648 non-null    object
 3   contractor_tier  660 non-null    object
 4   job_ids          660 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 30.9+ KB


In [9]:
df.head(5)

,job_title,job_description,list_tags,contractor_tier,job_ids
0,Tiktok followers and video likes,I opened an account and I’m trying to get it v...,"Advertising, Graphic Design, YouTube Developme...",Intermediate,0
54,Advisory work re google ads,We are looking for an experienced professional...,"Social Media Advertising, Social Media Managem...",Expert,54
86,Facebook Advertising For Real Estate Companies,Intro We run a specialised marketing agency ca...,"Advertising, Paid Media, Freelance Marketing, ...",Expert,86
114,SMM advertising specialist,I run a Seamoss gel company. I need an adverti...,"Logo Design, 3D Clothing Design",Expert,114
144,PART TIME - Ad Trafficker/Campaign Manager/Ad ...,***1 position open for 2 hrs. daily for the af...,"Advertisement, Adobe Photoshop, Adobe Illustra...",Intermediate,144


In [10]:
# drop any row that has a null value
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 648 entries, 0 to 64201
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   job_title        648 non-null    object
 1   job_description  648 non-null    object
 2   list_tags        648 non-null    object
 3   contractor_tier  648 non-null    object
 4   job_ids          648 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 30.4+ KB


In [11]:
list_tags = list(df['list_tags'])
all_tags = []
for tags in list_tags:
    parts = tags.strip().split(',')
    parts = [part.strip() for part in parts]
    all_tags.extend(parts)
all_tags = list(set(all_tags))
print(all_tags[:10])
print('number of tags: ', len(all_tags))

['Appointment Setting', 'Python', 'Organizational Design & Effectiveness', 'Customer Acquisition', 'DaVinci Resolve', 'System Security', 'Thai', 'Community Relations', 'Relationship Management', 'Project Plans']
number of tags:  1422


In [12]:
from nltk.corpus import stopwords

# clean job description
STEMMER = nltk.stem.porter.PorterStemmer()

# function which stop words and the words less than 2 characters
def remove_stop_words (text):
  result = []
  for token in gensim.utils.simple_preprocess(text):
    if token not in gensim.parsing.preprocessing.STOPWORDS and token not in nltk.corpus.stopwords.words('english'):
      result.append(token)

  return result

def preprocess(txt, stem=True):
    # convert all characters in the string to lower case
    txt = txt.lower()
    # remove non-english characters, punctuation and numbers
    txt = re.sub('[^a-zA-Z]', ' ', txt)
    # tokenize word
    txt = nltk.tokenize.word_tokenize(txt)
    # remove stop words
    txt = [w for w in txt if not w in nltk.corpus.stopwords.words('english')]
    
    if stem:
        # stemming
        txt = [STEMMER.stem(w) for w in txt]

    return ' '.join(txt)


def concat(lst):
  return ' '.join(lst)

df['jd_cleaned'] = df['job_description'].apply(lambda w: preprocess(w, stem=False))
df['jd_cleaned'] = df['jd_cleaned'].apply(lambda w: remove_stop_words(w)).apply(concat).astype(str)
df.head(5)

,job_title,job_description,list_tags,contractor_tier,job_ids,jd_cleaned
0,Tiktok followers and video likes,I opened an account and I’m trying to get it v...,"Advertising, Graphic Design, YouTube Developme...",Intermediate,0,opened account trying viral followers likes da...
54,Advisory work re google ads,We are looking for an experienced professional...,"Social Media Advertising, Social Media Managem...",Expert,54,looking experienced professional provide advis...
86,Facebook Advertising For Real Estate Companies,Intro We run a specialised marketing agency ca...,"Advertising, Paid Media, Freelance Marketing, ...",Expert,86,intro run specialised marketing agency called ...
114,SMM advertising specialist,I run a Seamoss gel company. I need an adverti...,"Logo Design, 3D Clothing Design",Expert,114,run seamoss gel company need advertising speci...
144,PART TIME - Ad Trafficker/Campaign Manager/Ad ...,***1 position open for 2 hrs. daily for the af...,"Advertisement, Adobe Photoshop, Adobe Illustra...",Intermediate,144,position open hrs daily afternoon shift pm cet...


In [13]:
df['job_info'] = df.apply(lambda row: row['contractor_tier'] + '-' + row['job_title'] + '-' + row['jd_cleaned'], axis=1)
df.head(5)

,job_title,job_description,list_tags,contractor_tier,job_ids,jd_cleaned,job_info
0,Tiktok followers and video likes,I opened an account and I’m trying to get it v...,"Advertising, Graphic Design, YouTube Developme...",Intermediate,0,opened account trying viral followers likes da...,Intermediate-Tiktok followers and video likes-...
54,Advisory work re google ads,We are looking for an experienced professional...,"Social Media Advertising, Social Media Managem...",Expert,54,looking experienced professional provide advis...,Expert-Advisory work re google ads-looking exp...
86,Facebook Advertising For Real Estate Companies,Intro We run a specialised marketing agency ca...,"Advertising, Paid Media, Freelance Marketing, ...",Expert,86,intro run specialised marketing agency called ...,Expert-Facebook Advertising For Real Estate Co...
114,SMM advertising specialist,I run a Seamoss gel company. I need an adverti...,"Logo Design, 3D Clothing Design",Expert,114,run seamoss gel company need advertising speci...,Expert-SMM advertising specialist-run seamoss ...
144,PART TIME - Ad Trafficker/Campaign Manager/Ad ...,***1 position open for 2 hrs. daily for the af...,"Advertisement, Adobe Photoshop, Adobe Illustra...",Intermediate,144,position open hrs daily afternoon shift pm cet...,Intermediate-PART TIME - Ad Trafficker/Campaig...


In [32]:
def minimum_preprocess(txt, stem=True):
    # convert all characters in the string to lower case
    txt = txt.lower()
    # remove non-english characters, punctuation and numbers
    txt = re.sub('[^a-zA-Z]', ' ', txt)
    return ' '.join(txt)

def concat_info(row):
    tier = row['contractor_tier']
    title = row['job_title']
    jd_cleaned = row['jd_minimum_cleaned']
    return f'We need to hire a {tier} position for {title}. The job description is as follow. {jd_cleaned}'

df['jd_minimum_cleaned'] = df['job_description'].apply(lambda w: preprocess(w, stem=False))
df['job_minimum_info'] = df.apply(lambda row: concat_info(row), axis=1)

In [33]:
df.head(3)

,job_title,job_description,list_tags,contractor_tier,job_ids,jd_cleaned,job_info,jd_summarized,jd_minimum_cleaned,job_minimum_info
0,Tiktok followers and video likes,I opened an account and I’m trying to get it v...,"Advertising, Graphic Design, YouTube Developme...",Intermediate,0,opened account trying viral followers likes da...,Intermediate-Tiktok followers and video likes-...,A new Tiktok account was created to attract in...,opened account trying get viral followers like...,We need to hire a Intermediate position for Ti...
54,Advisory work re google ads,We are looking for an experienced professional...,"Social Media Advertising, Social Media Managem...",Expert,54,looking experienced professional provide advis...,Expert-Advisory work re google ads-looking exp...,Experienced professional is looking for an exp...,looking experienced professional provide advis...,We need to hire a Expert position for Advisory...
86,Facebook Advertising For Real Estate Companies,Intro We run a specialised marketing agency ca...,"Advertising, Paid Media, Freelance Marketing, ...",Expert,86,intro run specialised marketing agency called ...,Expert-Facebook Advertising For Real Estate Co...,Abb marketing is a specialised marketing agenc...,intro run specialised marketing agency called ...,We need to hire a Expert position for Facebook...


# text summarize

In [34]:
import os
os.environ['TRANSFORMERS_CACHE'] = '/data/tungtx2/tmp/'

from transformers import pipeline

summarizer = pipeline("summarization", model="philschmid/bart-large-cnn-samsum")

In [35]:
def get_summarized_text(text):
    res = summarizer(text, clean_up_tokenization_spaces=True)[0]
    return res['summary_text']

In [37]:
df['jd_minimum_summarized'] = df['job_minimum_info'].apply(lambda text: get_summarized_text(text))
df.sample(5)

Your max_length is set to 142, but you input_length is only 45. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=22)
Your max_length is set to 142, but you input_length is only 118. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=59)
Your max_length is set to 142, but you input_length is only 60. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=30)
Your max_length is set to 142, but you input_length is only 58. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=29)
Your max_length is set to 142, but you input_length is only 30. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=15)
Your max_length is set to 142, but you input_length is only 63. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=31)
Your max_length is set to 142, but you input_length is only 46. You might conside

,job_title,job_description,list_tags,contractor_tier,job_ids,jd_cleaned,job_info,jd_summarized,jd_minimum_cleaned,job_minimum_info,jd_minimum_summarized
16084,Freelancer for E-commerce setup,We need a freelancer for an E-commerce project...,"Lead Generation, Sales, Market Research, Custo...",Expert,16084,need freelancer commerce project knowledge set...,Expert-Freelancer for E-commerce setup-need fr...,Freelancer for E-commerce setup-need freelance...,need freelancer e commerce project knowledge s...,We need to hire a Expert position for Freelanc...,Freelancer for E-commerce setup is needed. The...
26577,"Facebook and Instagram Ads Expert (create, set...",I am looking for someone who could - create my...,"Content SEO, Competitive Analysis, Tech & IT, ...",Expert,26596,looking create fb insta ad account set create ...,Expert-Facebook and Instagram Ads Expert (crea...,Facebook and Instagram ads are being created t...,looking someone could create fb insta ad accou...,We need to hire a Expert position for Facebook...,Facebook and Instagram Ads Expert is needed to...
64201,Intune / Endpoint Manager / Autopilot Expert,We are seeking a Microsoft Endpoint and Autopi...,"Network Penetration Testing, Network Security,...",Intermediate,64344,seeking microsoft endpoint autopilot contracto...,Intermediate-Intune / Endpoint Manager / Autop...,Microsoft endpoint autopilot contractor will c...,seeking microsoft endpoint autopilot contracto...,We need to hire a Intermediate position for In...,Microsoft needs to hire a project contractor f...
53346,Researcher/Writer For Black Finance Youtube Ch...,Hey we are looking for a researcher and script...,"Video Editing, Video Production, Education",Expert,53395,hey looking researcher scriptwriter business y...,Expert-Researcher/Writer For Black Finance You...,Researcher/Writer for Black Finance Youtube Ch...,hey looking researcher scriptwriter business y...,We need to hire a Expert position for Research...,The job description is as follows: Researcher/...
6425,Full Stack Python/Flask Expert,We're looking for a senior python/flask engine...,"Mechanical Engineering, SolidWorks, 2D Design,...",Intermediate,6425,looking senior python flask engineer implement...,Intermediate-Full Stack Python/Flask Expert-lo...,The company is looking for a senior python fla...,looking senior python flask engineer implement...,We need to hire a Intermediate position for Fu...,The company is looking for a senior python fla...


In [ ]:
df.sample(5)

,job_title,job_description,list_tags,contractor_tier,job_ids,jd_cleaned,job_info,jd_summarized
34670,WordPress website дизайнер программист,Обязанности: 1. Дизайн пользовательского интер...,"Android App Development, Web Development, Reac...",Expert,34692,wordpress html css wordpress wordpress wordpre...,Expert-WordPress website дизайнер программист-...,WordPress website has a theme forest theme for...
48075,English into Turkmen translators and editors n...,We are looking for native experienced linguist...,"Translation, English, French, Documentation",Intermediate,48101,looking native experienced linguists translate...,Intermediate-English into Turkmen translators ...,Intermediate-English into Turkmen translators ...
45150,Prodigies Bells - Unity App Update,Hey Jason -- we worked together back in 2021 o...,"Node.js, Agile Software Development, Front-End...",Intermediate,45176,hey jason worked instrument app called prodigi...,Intermediate-Prodigies Bells - Unity App Updat...,Prodigies Bells - Unity App Update is being de...
6847,Need android application upload on store,Hello Need developer for publish my appps on g...,"Java, AngularJS",Entry level,6847,hello need developer publish appps google play...,Entry level-Need android application upload on...,The entry level for Google Play Store is a Goo...
33198,Finding conferences,We need to find a list of all upcoming major c...,"Appointment Setting, Lead Generation, Communic...",Intermediate,33220,need list upcoming major conferences dc va md ...,Intermediate-Finding conferences-need list upc...,There are several upcoming major conferences i...


In [20]:
df.iloc[10]['jd_summarized']

'Facebook advertising expert is needed to create effective facebook advertising campaign. Fitness products send message information to send a message to the user.    Facebook adverts expert is wanted. For more information, go to: www.facebook.com/advertisingexpert.'

In [38]:
df.to_csv('jd_summarized.csv', index=False)

In [26]:
df.iloc[100]['job_description']

'Involve Mininet, SDN and flow insertion. Details will be provided on request. Involves developing python based applications'